In [ ]:
from operator import itemgetter


class HMM:
    def __init__(self,A,B,pi):
        self.O = None
        self.A = A
        self.B = B
        self.pi = pi
        self.mem = {}
        self.mem2 = {}
        if len(A) != len(B):
            raise ValueError("A and B must have same length")
        if len(A) != len(pi):
            raise ValueError("A and pi must have same length")
        
        
        
    def observe_seq(self,O):
        self.O = O
    def _alpha(self,t, j):
        A = self.A
        B = self.B
        pi = self.pi
        mem = self.mem
        O = self.O
        if (t,j) in mem:
            return mem[t,j]
        if t == 0:
            return pi[j]*B[j][O[t]]
        previous = 0
        for i in range(len(A)):
            previous += self._alpha(t - 1, i) * A[i][j]
        mem[t,j] = B[j][O[t]]* previous
        return mem[t,j]
    
    def _beta(self,t,i):
        A = self.A
        B = self.B
        
        mem2 = self.mem2
        O = self.O
        forward = 0
        if t -1 == len(O):
            return 1
        if (t,i) in mem2:
            return mem2[t,i]
        for j in range(len(A)):
           forward+= A[i][j]*B[j][O[t+1]]*self._beta(t+1,j)
        mem2[t,i] = forward
        return mem2[t,i]
        
    
    def prob(self,O):
        self.O = O
        return sum([self._alpha(len(O),i) for i in range(len(self.A))])
    
    def _viterbi(self,T,N):
        A = self.A
        B = self.B
        pi = self.pi
        O = self.O
        p_star = 1
        v_0 = max([(pi[i]*B[i][O[0]],i) for i in range(N)], key=itemgetter(0))
        i = v_0[1]
        v_path = [i]
        p_star = p_star*v_0[0]
        for t in range(1, T):
            v = max([(B[j][O[t]]*A[i][j],j) for j in range(N)],key=itemgetter(0))
            p_star = p_star * v[0]
            i = v[1]
            v_path.append(i)
        return p_star,v_path
    
    def viterbi(self):
        return self._viterbi(len(self.O),len(self.A))
    
    
    def _delta(self,t):
        A = self.A
        B = self.B
        pi = self.pi
        O = self.O
        def exp(i,j):
            normalize = sum([self._alpha(t,k)*self._beta(t,k) for k in range(len(A))])
            return self._alpha(t,i)*A[i][j]*B[j][O[t+1]]*self._beta(t+1,j)/normalize
        return exp

    def _gamma(self,t):
        A = self.A
        def exp(i):
            normalize = sum([self._alpha(t,k)*self._beta(t,k) for k in range(len(A))])
            return self._alpha(t,i)*self._beta(t,i)/normalize
        return exp
    
    def train(self,acc):
        A = [[[0]*len(self.A)] for _ in range(len(self.A))]
        B = [[[0]*len(self.B[0])] for _ in range(len(self.B))]
        for _ in range(acc):
            for i in range(len(self.A)):
                for j in range(len(self.A)):
                    A[i][j] = sum([ self._delta(t)(i,j) for t in range(len(self.O))])/sum([ self._delta(t)(i,k) for t in range(len(self.O)) for k in range(len(self.A))])
                for k in range(len(self.B[0])):
                    B[i][k] = sum([self._gamma(t)(i) for t in range(len(self.O)) if self.O[t] == k])/sum([self._gamma(t)(i) for t in range(len(self.O))])
        self.A = A
        self.B = B
        return A,B
                    
            
        
        
        
        
        